In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('project5').getOrCreate()

In [4]:
data = spark.read.csv('Dataset/smsspamcollection/SMSSpamCollection',inferSchema=True,sep='\t')

In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn('length',length(data['text']))

In [10]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [11]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [13]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [14]:
from pyspark.ml.feature import VectorAssembler

In [15]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [16]:
from pyspark.ml.classification import NaiveBayes

In [17]:
nb = NaiveBayes()

In [18]:
from pyspark.ml import Pipeline

In [20]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stop_remove,count_vec,idf,clean_up]
)

In [21]:
cleaner = data_prep_pipe.fit(data)

In [22]:
clean_data = cleaner.transform(data)

In [23]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [24]:
clean_data = clean_data.select('label','features')

In [25]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [26]:
train, test = clean_data.randomSplit([0.7,0.3])

In [27]:
spam_detector = nb.fit(train)

23/07/27 06:12:46 WARN DAGScheduler: Broadcasting large task binary with size 1164.9 KiB
23/07/27 06:12:48 WARN DAGScheduler: Broadcasting large task binary with size 1144.2 KiB


In [28]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [29]:
test_results = spam_detector.transform(test)

In [30]:
test_results.show()

23/07/27 06:13:12 WARN DAGScheduler: Broadcasting large task binary with size 1370.3 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-793.63330811549...|[1.0,6.4698488529...|       0.0|
|  0.0|(13424,[0,1,2,13,...|[-610.12623528999...|[1.0,6.0219058785...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-825.40587909246...|[1.0,4.6954375064...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-873.29505333110...|[1.0,1.2974684453...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-538.04333108631...|[1.0,7.6160670589...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-885.11949585422...|[1.0,5.7198542328...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-448.95223684724...|[1.0,3.1701436535...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-692.84427692239...|[1.0,2.6287073237...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-837.36131760964...|[1.0,1.8430301758...|       0.0|
|  0.0|(13424,[0

23/07/27 06:13:12 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/27 06:13:12 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [32]:
acc_eval = MulticlassClassificationEvaluator()

In [33]:
acc= acc_eval.evaluate(test_results)

23/07/27 06:13:44 WARN DAGScheduler: Broadcasting large task binary with size 1374.9 KiB


In [35]:
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9245455126089621
